# 2019-07-19: T0 Calculation

We would like to be able to compute the thermodynamic quantity T0 (T-zero) using pycalphad. This is currently (as of 0.8.3) a challenge because global minimization cannot be turned off in `equilibrium`. In this workaround example, we changed the calculation to use direct energy calculation with `calculate()` and bypass `equilibrium()` altogether to avoid global min issues.

This approach is pretty fast, but the downside is as you go to higher-order systems you will have to increase the pdens value (which increases RAM usage). The other disadvantage is accuracy, which is +- 10 K relative to the true (from the model) value, and it gets worse as you increase the number of components. This approximation should probably not be used with more than three independent components.

In [2]:
from pycalphad import Database, equilibrium, calculate, Model, ReferenceState, variables as v
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

dbf = Database("COST507.pycalphad.tdb") #Database
comps = ["TI", "V", "AL", "VA"] #Components
ph_p = 'BCC_A2' #Parent phase
ph_c = 'HCP_A3' #Child phase

## Minimization Algorithm
#Initialization
stp = 100 #Initial step size [K]
stp_red = 2 #Reduction factor of step value [1]
tol = .1 #Tolerance value [kJ/mol]
dG = 2*tol #Difference in molar Gibbs energy
T0 = 1273 #Initial temperature where only the parent phase is expected [K]
flag = 1 #[1]: Decreasing temperature [0]:Increasing temperature
comp_tol = 1e-2 # Tolerance for composition (mole fraction)

composition = {'V': 0.10, 'AL': 0.03}

#Optimization loop
while abs(dG) > tol:
    calc_p = calculate(dbf,comps,ph_p, T=T0, P=101325, pdens=50000)
    calc_c = calculate(dbf,comps,ph_c, T=T0, P=101325, pdens=50000)
    points_p = np.ones(len(calc_p.points), dtype=bool)
    points_c = np.ones(len(calc_c.points), dtype=bool)
    # Find configurations of desired composition
    for comp, val in composition.items():
        points_p &= (np.abs(calc_p.X.sel(component=comp) - val).values.squeeze() < comp_tol)
        points_c &= (np.abs(calc_c.X.sel(component=comp) - val).values.squeeze() < comp_tol)
    if not np.any(points_p):
        raise ValueError('Unable to find parent phase configurations at desired tolerance')
    if not np.any(points_c):
        raise ValueError('Unable to find child phase configurations at desired tolerance')
    idx_p = np.nonzero(points_p)
    idx_c = np.nonzero(points_c)
    # If multiple configurations at desired composition, choose minimum energy
    # Q: Is this reasonable? What about minimum of parent, max of child?
    GM_min_p = calc_p.GM.values.squeeze()[idx_p].min()
    GM_min_c = calc_c.GM.values.squeeze()[idx_c].min()
    #Check Gibbs energies at temperature and determine next step
    dG = GM_min_p - GM_min_c
    if dG < 0 and flag == 1: #G_P < G_D -> Reducing T
       T0 = T0-stp
       print("T:", np.floor(T0-273.15), "°C - reducing") #Print Status
    elif dG > 0 and flag == 0: #G_P > G_D -> Increasing T
       T0 = T0+stp
       print("T:", np.floor(T0-273.15), "°C - increasing") #Print Status
    elif dG < 0 and flag == 0: #G_P < G_D after T-increase-> Decreasing T with reduced stepsize
       stp = stp/stp_red #reduce step size
       flag = 1
       T0 = T0-stp
       print("T:", np.floor(T0-273.15), "°C - decreasing with SZ:",stp, "K") #Print Status
    elif dG > 0 and flag == 1: #G_P > G_D after T-reduction-> Increasing T with reduced stepsize
       stp = stp/stp_red #reduce step size
       flag = 0
       T0 = T0+stp
       print("T:", np.floor(T0-273.15), "°C - increasing with SZ:",stp, "K") #Print Status
#Result
print("The T0 temperature is", np.round(T0-273.15), "°C") #Print T0 [°C]

T: 899.0 °C - reducing
T: 799.0 °C - reducing
T: 699.0 °C - reducing
T: 599.0 °C - reducing
T: 499.0 °C - reducing
T: 549.0 °C - increasing with SZ: 50.0 K
T: 599.0 °C - increasing
T: 574.0 °C - decreasing with SZ: 25.0 K
T: 549.0 °C - reducing
T: 562.0 °C - increasing with SZ: 12.5 K
T: 574.0 °C - increasing
T: 568.0 °C - decreasing with SZ: 6.25 K
T: 562.0 °C - reducing
T: 565.0 °C - increasing with SZ: 3.125 K
T: 568.0 °C - increasing
T: 567.0 °C - decreasing with SZ: 1.5625 K
T: 567.0 °C - increasing with SZ: 0.78125 K
T: 568.0 °C - increasing
T: 568.0 °C - decreasing with SZ: 0.390625 K
T: 568.0 °C - increasing with SZ: 0.1953125 K
T: 568.0 °C - decreasing with SZ: 0.09765625 K
T: 568.0 °C - reducing
The T0 temperature is 568.0 °C
